# 📊 Telecom X – Parte 2: Predicción de Cancelación (Churn)
Este notebook desarrolla modelos de clasificación para predecir la cancelación de clientes.
La información fue preprocesada en la Parte 1 del proyecto.


In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report, confusion_matrix, ConfusionMatrixDisplay
from imblearn.over_sampling import SMOTE
import warnings
warnings.filterwarnings('ignore')
sns.set(style='whitegrid')

## 📥 Carga del dataset limpio

In [ ]:
# Suponemos que el dataset limpio fue exportado desde la Parte 1 como CSV
df = pd.read_csv('TelecomX_datos_limpios.csv')  # ← Reemplaza con la ruta real si es necesario
df.head()

## ⚙️ Preprocesamiento de datos

In [ ]:
# Eliminar columnas irrelevantes
df = df.drop(columns=['ID_Cliente'], errors='ignore')

# Convertir columna objetivo a binaria si no lo está
df['Evasion'] = df['Evasion'].map({'Yes': 1, 'No': 0}) if df['Evasion'].dtype == 'object' else df['Evasion']

# Codificación de variables categóricas
df_encoded = pd.get_dummies(df, drop_first=True)

# Separar variables y objetivo
X = df_encoded.drop('Evasion', axis=1)
y = df_encoded['Evasion']

## 🔄 Balanceo de clases con SMOTE

In [ ]:
smote = SMOTE(random_state=42)
X_bal, y_bal = smote.fit_resample(X, y)
y_bal.value_counts()

## 📚 División de datos en entrenamiento y prueba

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X_bal, y_bal, test_size=0.3, random_state=42, stratify=y_bal)

## 📏 Normalización para modelos que lo requieren

In [ ]:
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

## 🤖 Modelo 1: Regresión Logística

In [ ]:
log_model = LogisticRegression(max_iter=1000)
log_model.fit(X_train_scaled, y_train)
y_pred_log = log_model.predict(X_test_scaled)

print(classification_report(y_test, y_pred_log))
ConfusionMatrixDisplay.from_predictions(y_test, y_pred_log, cmap='Blues')
plt.title('Matriz de Confusión - Regresión Logística')
plt.show()

## 🌳 Modelo 2: Random Forest

In [ ]:
rf_model = RandomForestClassifier(random_state=42)
rf_model.fit(X_train, y_train)
y_pred_rf = rf_model.predict(X_test)

print(classification_report(y_test, y_pred_rf))
ConfusionMatrixDisplay.from_predictions(y_test, y_pred_rf, cmap='Greens')
plt.title('Matriz de Confusión - Random Forest')
plt.show()

## 📌 Importancia de Variables (Random Forest)

In [ ]:
importances = pd.Series(rf_model.feature_importances_, index=X.columns)
importances.nlargest(15).sort_values().plot(kind='barh')
plt.title('Top 15 Variables más importantes')
plt.xlabel('Importancia')
plt.tight_layout()
plt.show()

## ✅ Conclusiones y Recomendaciones

- Ambos modelos tuvieron un buen desempeño, siendo Random Forest más robusto.
- Las variables clave fueron el tipo de contrato, método de pago, permanencia y cargos mensuales.
- Recomendaciones estratégicas:
  1. Incentivar contratos largos.
  2. Atender más a los clientes nuevos.
  3. Promover pagos automáticos o electrónicos.
  4. Revisar experiencia de clientes con cargos altos.